In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from datetime import datetime
import os
import glob

In [29]:
#subreddits = ['Minecraft','minecraftsuggestions','MinecraftDungeons','MinecraftServer']
#subreddits = ['thesims','Sims4','TheSimsBuilding','Sims3', 'thesims4']
#subreddits = ['Overwatch', 'OverwatchLeague', 'Competitiveoverwatch', 'Competitiveoverwatch']
#subreddits = ['skyrim', 'skyrimmods', 'SkyrimModsXbox', 'kyrimTogether']
subreddits = ['roblox','RobloxDevelopers', 'RobloxAvatars', 'RobloxHelp']
#subreddits = ["GTA", "GTAOnline", "GTA6", "GrandTheftAutoV", "GTAV", "GTAV_Mods"]
#subreddits = ['leagueoflegends',  'summonerschool', 'LeagueConnect']
#subreddits = ['gaming']
#subreddits = ['CallOfDuty']
#subreddits = ['Valorant']

name = "roblox"

In [ ]:
# Setup ChromeDriver
service = Service(executable_path="C:/Windows/chromedriver.exe")
options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36")
driver = webdriver.Chrome(service=service, options=options)

# Store all posts here
all_posts = []
found_2025 = 0
max_posts = 1000


for subreddit in subreddits:
    if found_2025 >= max_posts:
        break

    print(f"\nScraping r/{subreddit} for 2025 posts...")

    driver.get(f"https://www.reddit.com/r/{subreddit}/new/")
    time.sleep(8)

    scroll_count = 0

    while scroll_count < max_scrolls and found_2025 < max_posts:
        new_2025_posts = 0
        posts = driver.find_elements(By.TAG_NAME, "shreddit-post")

        for post in posts:
            if found_2025 >= max_posts:
                break

            post_id = post.get_attribute("id")
            if not post_id or post_id in seen_ids:
                continue
            seen_ids.add(post_id)

            title = post.get_attribute("post-title") or "No title"
            timestamp = post.get_attribute("created-timestamp")
            if not timestamp:
                continue

            try:
                dt = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%f%z")
                if dt.year != 2025:
                    continue

                created_utc = dt.strftime('%Y-%m-%d %H:%M:%S')
                all_posts.append({
                    "subreddit": subreddit,
                    "year": dt.year,
                    "month": dt.month,
                    "text": title,
                    "created_utc": created_utc,
                })
                found_2025 += 1
                new_2025_posts += 1
                print(f"{title[:60]}")

            except Exception as e:
                print(f"Error parsing timestamp: {e}")

        scroll_count += 1
        print(f"Scroll {scroll_count} — Found {new_2025_posts} new 2025 posts — Total: {found_2025}")

        if new_2025_posts == 0 or found_2025 >= max_posts:
            print("No new 2025 posts found or max reached. Stopping scroll.")
            break

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

# Save to single CSV
df = pd.DataFrame(all_posts)
filename = f"{name}-2025.csv"
df.to_csv(filename, index=False, encoding='utf-8')
print(f"\nSaved {len(df)} posts to {filename}")

# Close browser
driver.quit()


Scraping r/roblox for 2025 posts...
Scroll 1 — Found 0 new 2025 posts — Total: 0
No new 2025 posts found or max reached. Stopping scroll.

Scraping r/RobloxDevelopers for 2025 posts...


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=138.0.7204.101)
Stacktrace:
	GetHandleVerifier [0x0x7ff7557acda5+78885]
	GetHandleVerifier [0x0x7ff7557ace00+78976]
	(No symbol) [0x0x7ff755569bca]
	(No symbol) [0x0x7ff755541e71]
	(No symbol) [0x0x7ff7555f039e]
	(No symbol) [0x0x7ff755610b42]
	(No symbol) [0x0x7ff7555e8963]
	(No symbol) [0x0x7ff7555b16b1]
	(No symbol) [0x0x7ff7555b2443]
	GetHandleVerifier [0x0x7ff755a84eed+3061101]
	GetHandleVerifier [0x0x7ff755a7f33d+3037629]
	GetHandleVerifier [0x0x7ff755a9e592+3165202]
	GetHandleVerifier [0x0x7ff7557c730e+186766]
	GetHandleVerifier [0x0x7ff7557ceb3f+217535]
	GetHandleVerifier [0x0x7ff7557b59b4+114740]
	GetHandleVerifier [0x0x7ff7557b5b69+115177]
	GetHandleVerifier [0x0x7ff75579c368+10728]
	BaseThreadInitThunk [0x0x7ffd946d259d+29]
	RtlUserThreadStart [0x0x7ffd9636af78+40]


In [ ]:
folder_path = "2025 data" 
output_file = "merged_2025data.csv"

csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
dfs = []

for file in csv_files:
    df = pd.read_csv(file)
    if 'year' in df.columns:
        df = df[df['year'] == 2025]
    df = df.head(200)
    if not df.empty:
        dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)
merged_df.to_csv(output_file, index=False, encoding='utf-8')
print(f"Merged {len(csv_files)} files into {output_file} with {len(merged_df)} total rows (max 200 from each).")

Merged 10 files into 2025data.csv with 2000 total rows (max 200 from each).
